# Taller 6 - Hadoop - HDFS

Juan Navarro, <jsnavarroa@unal.edu.co>



# Instalación

* Install Hadoop

```bash
cd "${HOME}/worskpace/BDA"

git clone https://github.com/jsnavarroa/docker-hadoop.git
cd docker-hadoop

# Install docker-compose
conda create --name py3 python=3.6
conda activate py3
conda install -c conda-forge docker-compose

# Run local
docker-compose -f docker-compose-local.yml build
docker-compose -f docker-compose-local.yml up -d
```

* Open the url http://localhost:58188/.
  * NameNode http://localhost:8088/

* Copy to container and Upload data to hdfs. The hdfs definition is in
docker-hadoop/hadoop.env

```bash
cd "${HOME}/worskpace/BDA/ADB/"

docker cp Hadoop/ namenode:/Example/

# Upload to hdfs
docker exec -it namenode bash
cd /Example

hadoop fs -mkdir hdfs://namenode:9000/bda
hadoop fs -put ./data/purchases.txt hdfs://namenode:9000/bda
hadoop fs -ls hdfs://namenode:9000/bda

hadoop fs -tail hdfs://namenode:9000/bda/purchases.txt
```

* Run the mapper and reducer written in python. [Hadoop Streaming](https://hadoop.apache.org/docs/r2.7.1/hadoop-streaming/HadoopStreaming.html)

```bash
docker exec -it namenode bash
cd /Example/src/store-totals

# Run job
pip3 install mrjob

python store_totals.py -r hadoop hdfs://namenode:9000/bda/purchases.txt > /Example/data/mrjob_out.txt
```
